In [2]:
PDF_FILE = "/teamspace/studios/this_studio/Computer Networking A Top-Down Approach, 8th Edition by James F. Kurose, Keith W. Ross-Pearson-9780136681557 (2).pdf"

# We'll be using Llama 3.1 8B for this example.
MODEL = "llama3.1"
     

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 177.4 MB/s eta 0:00:0000:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0


In [4]:
!pip install langchain langchain-community langchain_ollama datasets faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 112.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 29.5 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
ds = load_dataset("juasdexter/interview_questions")

README.md:   0%|          | 0.00/224 [00:00<?, ?B/s]

interview_questions.csv:   0%|          | 0.00/257k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/602 [00:00<?, ? examples/s]

In [7]:

print(ds['train'][0])

{'qid': 1, 'language': 'Java', 'level': 'Easy', 'question': 'What is Java?', 'answer': 'Java is a high-level, class-based, object-oriented programming language.'}


In [8]:
lines = []

# Iterate through each item in the dataset
for x in ds['train']:
    # Append the formatted string to the list
    lines.append(f"language:{x['language']} question:{x['question']} answer:{x['answer']}")

    # Write all collected lines to a file
with open('data.txt', 'w') as f:
        # Join all the lines with a newline character and write to the file
        f.write("\n".join(lines))

In [10]:
with open('data.txt','r') as f:
    pages=f.read()
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
chunks = splitter.split_text(pages)

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL)  
vectorstore = FAISS.from_texts(chunks, embeddings)

In [11]:

from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("Who is the president of the United States?")

AIMessage(content='As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-20T13:48:03.59314216Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2363705788, 'load_duration': 22164380, 'prompt_eval_count': 19, 'prompt_eval_duration': 24000000, 'eval_count': 65, 'eval_duration': 2316000000}, id='run-3253690c-9bd5-4ebe-a628-7efdc2bde7fa-0', usage_metadata={'input_tokens': 19, 'output_tokens': 65, 'total_tokens': 84})

In [12]:

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("Who is the president of the United States?"))

As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.


In [28]:

from langchain.prompts import PromptTemplate

template = """
You are an assistant that extracts skills like JAVA, C++, AWS, React etc from job descriptions by analyzing the specific skill required
in job descriptions based on a given context. 
Extract as many technical skills as you can based on the context. If you can't extract, reply "I don't know".

Be as concise as possible,just giving the names of skills, and go straight to the point.

Context: {context}

JobDescription: {jobDescription}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", jobDescription="Here is a question"))


You are an assistant that extracts skills like JAVA, C++, AWS, React etc from job descriptions by analyzing the specific skill required
in job descriptions based on a given context. 
Extract as many technical skills as you can based on the context. If you can't extract, reply "I don't know".

Be as concise as possible,just giving the names of skills, and go straight to the point.

Context: Here is some context

JobDescription: Here is a question



In [29]:
chain = prompt | model | parser


In [30]:

from operator import itemgetter
retriever = vectorstore.as_retriever()

chain = (
    {
        "context": itemgetter("jobDescription") | retriever,
        "jobDescription": itemgetter("jobDescription"),
    }
    | prompt
    | model
    | parser
)
     

In [31]:
jobDescription = '''GTechSources is a leading, multidimensional product-based software house and a certified technology company in the U.S., dedicated to assisting businesses on their growth journey. We specialize in developing modern marketing strategies, innovative web solutions, and impactful designs.

Responsibilities:

Design, develop, and maintain high-performance, scalable backend systems using Java and Spring Boot.
Collaborate with front-end developers, designers, and product managers to deliver end-to-end solutions.
Write clean, maintainable, and efficient code in accordance with industry best practices.
Implement RESTful APIs and ensure seamless integration with third-party services.
Participate in code reviews, testing, and deployment processes to ensure quality.
Troubleshoot, debug, and upgrade existing systems as needed.
Contribute to architectural discussions and provide insights on best practices.
Ensure the security, scalability, and reliability of backend systems.
Stay updated with the latest industry trends and technologies.
Requirements:

Minimum Bachelor's degree in Computer Science or a related field.
Proven experience as a Backend Developer, with a focus on Java Spring Boot.
Strong understanding of Object-Oriented Programming (OOP) principles.
Excellent communication and teamwork skills.
What We Offer:

Market Competitive Salary
Weekends are Off (Saturday and Sunday)!
22 Paid Holidays
Medical Reimbursement
Probationary and Annual Increment
Opportunities for professional growth and development.
A collaborative and innovative work environment.
Working Hours:

12:00 PM – 9:00 PM
Job Type: Full-time

Ability to commute/relocate:

Lahore: Reliably commute or planning to relocate before starting work (Preferred)
Application Question(s):

Our standard operating hours are from 12 PM to 9 PM, Monday to Friday. Would you be willing to work during these hours?
Education:

Bachelor's (Preferred)
Experience:

Java Spring Boot: 2 years (Preferred)
Location:

Lahore (Preferred)'''
print(f"Answer: {chain.invoke({'jobDescription': jobDescription})}")
print("*************************\n")
skills=chain.invoke({'jobDescription': jobDescription})

Answer: Here are the technical skills extracted from the job description:

1. Java
2. Spring Boot
3. RESTful APIs
4. Object-Oriented Programming (OOP)
5. Code reviews
6. Testing
7. Deployment
8. Troubleshooting
9. Debugging
*************************



'Here are the technical skills extracted from the job description:\n\n1. Java\n2. Spring Boot\n3. RESTful APIs\n4. Object-Oriented Programming (OOP)\n5. Code reviews\n6. Testing\n7. Deployment\n8. Troubleshooting\n9. Debugging'

In [54]:

from langchain.prompts import PromptTemplate

template2 = """
You are an assistant that receives technical skills input and generates quizzes based on given context. Each question
in quiz should have 4 questions, 3 wrong answers and 1 correct answer.
Generate as many questions as you can based on the given context only relevant to the skills. If a skill is not
relevant to context do not generate a question for that. If you can't generate, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Skills: {skills}
"""

prompt2 = PromptTemplate.from_template(template2)
print(prompt2.format(context="Here is some context", skills="Here is a skills"))


You are an assistant that receives technical skills input and generates quizzes based on given context. Each question
in quiz should have 4 questions, 3 wrong answers and 1 correct answer.
Generate as many questions as you can based on the given context only relevant to the skills. If a skill is not
relevant to context do not generate a question for that. If you can't generate, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Here is some context

Skills: Here is a skills



In [55]:

chain = (
    {
        "context": itemgetter("skills") | retriever,
        "skills": itemgetter("skills"),
    }
    | prompt2
    | model
    | parser
)
     

In [56]:
questions=chain.invoke({'skills': skills})

In [57]:
print(questions)

Here are the generated questions based on the given context and skills:

**Java**

1. What is the difference between an 'Error' and an 'Exception' in Java?
	* A) An Error represents a condition that can be handled by the program.
	* B) An Exception represents a serious problem that occurs at runtime.
	* C) Both are used to represent conditions that can be handled by the program.
	* D) Neither is used to represent conditions that can be handled by the program.

Answer: B) An Exception represents a serious problem that occurs at runtime.

2. What is the purpose of the 'finally' block in a 'try-catch-finally' statement?
	* A) To specify code that will be executed only if an exception is thrown.
	* B) To specify code that will be executed regardless of whether an exception is thrown or not.
	* C) To specify code that will be executed only if the exception is caught.
	* D) To specify code that will be executed only if the exception is not caught.

Answer: B) To specify code that will be exe

In [49]:
skills

'Here are the technical skills extracted from the job description:\n\n1. Java\n2. Spring Boot\n3. RESTful APIs\n4. Object-Oriented Programming (OOP)\n5. Code reviews\n6. Testing\n7. Deployment\n8. Troubleshooting\n9. Debugging'